In [1]:
import cv2
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets.fashion_mnist import load_data
from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Input, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

%matplotlib inline

In [2]:
def model_save( model, history, filename ):
    base_path = './Results/NN/'
    h5 = base_path + 'Models/' + filename + '.h5'
    weights = base_path + 'weights/' + filename + '-weights.h5'
    pkl = base_path + 'history/'+ filename +'.pkl'
    try:
        # 保存
        model.save( h5 )
        model.save_weights( weights )

        with open( pkl, 'wb' ) as h_file:
            pickle.dump(history.history, h_file)
        print("学習結果の保存が完了しました。")
    except:
        print("保存に失敗しました。")

def img_reshape(img_list):
    new_image = np.empty((len(img_list), 64, 64), dtype='uint8')
    for i, _ in enumerate(img_list):
        new_image[i] = cv2.resize(img_list[i], dsize=(64, 64))
    return new_image

random_state = 60

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

class_num = len(class_names)

(x_train, y_train), (x_test, y_test) = load_data()
x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.9, random_state=random_state)
_, x_test, _, y_test = train_test_split(x_test, y_test, test_size=0.1, random_state=random_state)

x_train = img_reshape(x_train)
x_test = img_reshape(x_test)

x_train = x_train.astype('float32')
x_train = x_train / 255.0
x_test = x_test.astype('float32')
x_test = x_test / 255.0

# one-hot表現に変換
y_train = to_categorical( y_train, class_num )
y_test = to_categorical( y_test, class_num )

In [3]:
def create_model(class_num):
    input_tensor = Input( shape=(64, 64, 1) )
    vgg = VGG16( include_top=False, input_tensor=input_tensor, weights=None )
    x = vgg.output
    x = Flatten()(x)
    x = Dense(class_num, activation="softmax")(x)
    return Model( inputs=vgg.inputs, outputs=x )

model = create_model(class_num)
model.summary()

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, verbose=1)
model = create_model( class_num )
print("コンパイルします")
model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
print("学習を開始します")
history = model.fit( x=x_train, y=y_train, batch_size=4,
                     epochs=200, verbose=1, validation_data=(x_test, y_test),
                     callbacks=[earlystopping]
                   )
print("モデルを保存します")
model_save(model, history, 'vgg16-data={}-epochs=200'.format(len(x_train)))

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)      

In [4]:
model = create_model(class_num)
model.summary()

earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=1)
print("コンパイルします")
model.compile(optimizer=Adam( learning_rate=1e-4 ), loss='categorical_crossentropy', metrics=['accuracy'])
print("学習を開始します")
history = model.fit( x=x_train, y=y_train, batch_size=4,
                     epochs=10, verbose=1, validation_data=(x_test, y_test),
                     callbacks=[earlystopping]
                   )
print("モデルを保存します")
model_save(model, history, 'vgg16-data={}-epochs=10'.format(len(x_train)))

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)      